# Financial Data Collection and Storage #

## Env Setup ##

In [1]:
# lib imports
import requests
import pandas as pd
import yfinance as YF
from fredapi import Fred
import mysql.connector as SQL


## Data Collection ##

In [2]:
# define key variables
fred_key = "7c1f3f043ef85e198d4c6c17e0d8109e"
tickers = [
    "AAPL",  # Apple
    "MSFT",  # Microsoft
    "JPM",   # JPMorgan Chase
    "WMT",   # Walmart
    "META",  # Meta Platforms
    "AMZN",  # Amazon
    "LMT",   # Lockheed Martin
    "GOOGL", # Alphabet
    "GS",    # Goldman Sachs
    "XOM",   # ExxonMobil
    "CVX",   # Chevron
    "PG",    # Procter & Gamble
    "KO",    # Coca-Cola
    "FDX",   # FedEx
    "UNP",   # Union Pacific
    "JNJ",   # Johnson & Johnson
    "PFE"    # Pfizer
]


# func to obtain stock data info from yahoo finance
def get_tickers(tickers=[], period='1d'):
    # get tickers from yahoo finance
    table = pd.DataFrame()
    for ticker in tickers:
        data = YF.Ticker(ticker)
        df = data.history(period=period)
        df['ticker'] = ticker
        table = pd.concat([table, df])
    return table

In [3]:
# pull stocks data
stocks = get_tickers(tickers, period='max')
stocks.reset_index(inplace=True)

In [4]:
# pull fred data
fred_api = Fred(fred_key)
cpi = fred_api.get_series('CPIAUCNS')
unrate = fred_api.get_series("Unrate")
csi = fred_api.get_series("UMCSENT")
dff = fred_api.get_series("DFF")
m30 = fred_api.get_series('MORTGAGE30US')

In [5]:
# join fred data
fred = pd.DataFrame(pd.concat([cpi, unrate, csi, dff, m30], axis=1)).reset_index()
fred.rename(columns={'index': 'Date', 0: 'cpi', 1: 'unrate', 2:'csi', 3: 'dff', 4: '30Y_mort'}, inplace=True)

In [6]:
# Convert stock dates to naive datetime (removing timezone)
stocks['Date'] = stocks['Date'].dt.tz_localize(None)

# Convert fred dates to ensure consistency (though likely already naive)
fred['Date'] = fred['Date'].dt.tz_localize(None)

# merge data on date
fi_df = pd.merge(stocks, fred, on='Date', how='left')

# backfill monthly indicators to prevent NaN values
fi_df[['cpi', 'unrate', 'csi', 'dff', '30Y_mort']] = fi_df[['cpi', 'unrate', 'csi', 'dff', '30Y_mort']].ffill()
fi_df[['cpi', 'unrate', 'csi', 'dff', '30Y_mort']] = fi_df[['cpi', 'unrate', 'csi', 'dff', '30Y_mort']].bfill()

# ensure consistency
fi_df.columns = fi_df.columns.str.lower()
# place ticker at pos 0
fi_df.insert(0, 'ticker', fi_df.pop('ticker'))

In [7]:
fi_df

,ticker,date,open,high,low,close,volume,dividends,stock splits,cpi,unrate,csi,dff,30y_mort
0,AAPL,1980-12-12,0.098726,0.099155,0.098726,0.098726,469033600,0.0,0.0,89.100,7.2,72.4,19.44,14.83
1,AAPL,1980-12-15,0.094005,0.094005,0.093575,0.093575,175884800,0.0,0.0,89.100,7.2,72.4,19.62,14.83
2,AAPL,1980-12-16,0.087136,0.087136,0.086707,0.086707,105728000,0.0,0.0,89.100,7.2,72.4,20.45,14.83
3,AAPL,1980-12-17,0.088853,0.089282,0.088853,0.088853,86441600,0.0,0.0,89.100,7.2,72.4,20.27,14.83
4,AAPL,1980-12-18,0.091429,0.091858,0.091429,0.091429,73449600,0.0,0.0,89.100,7.2,72.4,20.74,14.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195864,PFE,2025-03-24,26.100000,26.219999,25.889999,26.139999,37944400,0.0,0.0,315.493,4.2,71.8,4.33,6.67
195865,PFE,2025-03-25,26.160000,26.200001,25.510000,25.549999,47607600,0.0,0.0,315.493,4.2,71.8,4.33,6.67
195866,PFE,2025-03-26,25.520000,25.650000,25.129999,25.209999,49487400,0.0,0.0,315.493,4.2,71.8,4.33,6.67
195867,PFE,2025-03-27,25.200001,25.230000,24.830000,25.010000,51302000,0.0,0.0,315.493,4.2,71.8,4.33,6.65


## Data Storage ##

In [8]:
# set up connection
conn = SQL.connect(host='localhost',
                   user='root',
                   password='root',
                   database='fi_db')

cursor = conn.cursor()

In [9]:
cursor.execute("DROP DATABASE IF EXISTS fi_db;")

In [10]:
# create db and activate db
cursor.execute("CREATE DATABASE IF NOT EXISTS fi_db;")
cursor.execute("USE fi_db;")

# create table with schema
cursor.execute("""
               CREATE TABLE IF NOT EXISTS hist(
                id INT AUTO_INCREMENT PRIMARY KEY,
                ticker VARCHAR(10),
                date DATETIME,
                open FLOAT,
                high FLOAT,
                low FLOAT,
                close FLOAT,
                volume FLOAT,
                dividends FLOAT,
                `stock splits` FLOAT,
                cpi FLOAT,
                unrate FLOAT,
                csi FLOAT,
                dff FLOAT,
                30y_mort FLOAT);
                """)

In [11]:
# push fi_df to sql
for idx, row in fi_df.iterrows():
    cursor.execute("""
                   INSERT INTO fi_db.hist (ticker, date, open, high, low,
                   close, volume, dividends, `stock splits`, cpi, unrate, csi, dff, 30y_mort)
                   VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                   """,
                   (row['ticker'], row['date'], row['open'], row['high'], row['low'],
                    row['close'], row['volume'], row['dividends'], row['stock splits'],
                    row['cpi'], row['unrate'], row['csi'], row['dff'], row['30y_mort'])
            )

In [12]:
# finalize the transfer
try:
    conn.commit()
    cursor.close()
    conn.close()
    print("Data successfully transferred!")
except Exception as e:
    print(e)

Data successfully transferred!
